In [ ]:
from DDQNAgent import Agent
from Datasets import CassavaLeafDataset
from QNetwork import QNetwork
from Memory import Memory

In [ ]:
class Config:
    def __init__(self, new_class, minor_classes, train_step, restore_model_path, gamma, learning_rate, batch, 
                epsilon_range, epsilon_polynomial_decay_step, target_soft_update, target_update_step, save_term, evaluation_term,
                show_phase):
        self.new_class = new_class
        self.minor_classes = minor_classes
        self.train_step = train_step
        self.restore_model_path = restore_model_path
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.batch = batch
        self.epsilon_range = epsilon_range
        self.epsilon_polynomial_decay_step = epsilon_polynomial_decay_step
        self.target_soft_update = target_soft_update
        self.target_update_step = target_update_step
        self.save_folder = '.model'
        self.save_term = save_term
        self.evaluation_term = evaluation_term
        self.show_phase = show_phase

config = Config({0:[0], 1:[1], 2:[2], 3: [3], 4:[4]}, [0, 1, 2], 120000, '', 0.1, 0.00025, 32, [0.01, 1], 120000, 1., 1000, 120000, 1000, 'Validation')

In [ ]:
dataset = CassavaLeafDataset(image_size = (128, 128), batch_size=64)

No of total samples in dataset and their distribution:  (array([0, 1, 2, 3, 4], dtype=int64), array([ 1087,  2189,  2386, 13158,  2577], dtype=int64))

Reward for each class.
	- Class 0 : 0.781136
	- Class 1 : 0.387891
	- Class 2 : 0.355865
	- Class 3 : 0.064531
	- Class 4 : 0.329489
Label is  [0 1 2 3 4]
Labels with count {0: 1087, 1: 2189, 2: 2386, 3: 13158, 4: 2577}


In [ ]:
q_network = QNetwork(config, 128, 'complex')
memory = Memory()
agent = Agent(q_network, dataset, memory, config)
agent.train()

C:\Users\hassa\OneDrive\Desktop\AI Degree\Knowledge Representation and Reasonins\DRL-on-Imbalanced-Data\network.py:90: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  x = tf.compat.v1.layers.conv2d(self.state, filters=32, kernel_size=3, strides=2, padding='same', activation=tf.nn.relu)
C:\Users\hassa\OneDrive\Desktop\AI Degree\Knowledge Representation and Reasonins\DRL-on-Imbalanced-Data\network.py:91: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  x = tf.compat.v1.layers.conv2d(x, filters=32, kernel_size=3, strides=2, activation=tf.nn.relu)
C:\Users\hassa\OneDrive\Desktop\AI Degree\Knowledge Representation and Reasonins\DRL-on-Imbalanced-Data\network.py:92: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In 

Starting Training.......
